In [1]:
import re

In [2]:
import pandas as pd
import numpy as np

def trim_resin_code(resin):
    return resin.str[5::]
def convert_wo(series):
    return str(series)

def compile_resin(df):
    resins=[]
    for resin in df[5:16:2].tolist():
        resins.append(resin)
    resins = [x for x in resins if type(x) != float]
    return resins

def compile_recipe(df):
    recipe=[]
    for r in df[6:17:2].tolist():
        recipe.append(str(r)[:-4])
    recipe = [x for x in recipe if x != '']
    return recipe
def clean_up(df):
    return df.drop(columns =['R1','R1 %','R2','R2 %','R3','R3 %','R4','R4 %','R5','R5 %','R6','R6 %','R1 kgs','R2 kgs','R3 kgs','R4 kgs','R5 kgs','R6 kgs','P1 kgs','P2 kgs'] )
def GSsearch_data():
    WO = pd.read_csv(r'C:\Users\bzhang\Desktop\Projects\CoatingDataManager\CoatingDataReport1.csv',low_memory=False)
    WO['ProductionDate'] = pd.to_datetime(WO['ProductionDate'])
    Features = ['Machine','Order','Product Code','Production Date','Qty','R1','R1 %','R2','R2 %','R3','R3 %','R4','R4 %','R5','R5 %','R6','R6 %', 'Run Time','OP Run Time','Setup Time','Maintenance Down Time','Operator','Total Production','Kg/Hr','Efficiency','SerialNbr','Construction','R1 kgs','R2 kgs','R3 kgs','R4 kgs','R5 kgs','R6 kgs','P1 kgs','P2 kgs','Paper Code 1','Paper Code 2']
    FeatureDict = dict(zip(WO.columns.values.tolist(),Features))
    WO = WO.rename(columns = FeatureDict)
    WO.Order = WO.Order.apply(convert_wo)
    WO.Machine = WO.Machine.str[3::]
    WO.iloc[:,5:17:2] = WO.iloc[:,5:17:2].apply(trim_resin_code,axis = 1)
    WO = WO.drop(WO[WO['Qty']<=0].index)
    return WO

WO = GSsearch_data()
WO['Resins'] = WO.apply(compile_resin, axis = 1).astype(str)
WO['Recipe'] = WO.apply(compile_recipe, axis = 1).astype(str)
kgslist = ['R1 kgs','R2 kgs','R3 kgs','R4 kgs','R5 kgs','R6 kgs','P1 kgs','P2 kgs']
for item in kgslist:
    WO[item].fillna(0,inplace = True)
    
WO = clean_up(WO)

In [3]:
WO.drop_duplicates(subset = ['SerialNbr'],keep = 'first',inplace = True)

In [4]:
def cleanConstruction(cons):
    if ('POLY' in cons) ==True:
        return cons.replace(',  POLY OVER PRINT','')
    else:
        return cons.replace(',  ,','')

In [5]:
WO['Construction']=WO['Construction'].astype(str)

In [6]:
WO['Paper Code 2'].fillna('None',inplace = True)

In [7]:
def get_web_width1(PC1):
    try:
        l = str(PC1).split('-')
        if l[0] == '18FJ':
            return 1.2
        if (l[0] in ['09WP','18WP','09IM'])==True:
            try:
                WP = str(WO[WO['Product Code'] ==PC1]['Paper Code 1'].iloc[0])
                l1 = WP.split('-')
                return float(l1[1][3:7])/1000
            except IndexError:
                return 0            
        else:
            return float(l[1][3:7])/1000
    except:
        return 'error'

In [8]:
def get_web_width2(PC2):
    try:
        if PC2 != 'None':
            l = str(PC2).split('-')
            if (l[0] in ['09WP','18WP','09IM'])==True:
                try:
                    WP = str(WO[WO['Product Code'] ==PC2]['Paper Code 1'].iloc[0])
                    l1 = WP.split('-')
                    return float(l1[1][3:7])/1000
                except IndexError:
                    return 0            
            else:
                return float(l[1][3:7])/1000
        else:
            return np.nan
    except:
        return 'error'

In [9]:
WO['Web Width 1'] = WO['Paper Code 1'].apply(get_web_width1)

In [10]:
WPlist = WO.loc[WO['Web Width 1']==0]['Product Code'].unique().tolist()

In [11]:
WPwidth = [1.04,1.04,1.04,1.04,1.04,0.91,0.64,0.91,0.91,0.91,0.91,1.14]
WPdict = dict(zip(WPlist,WPwidth))
WPdict['18WP-DIC-07716-15']=1.04

In [12]:
WO['Web Width 1'] = WO.apply(lambda x: WPdict[x['Product Code']] if x['Web Width 1']==0 else x['Web Width 1'],axis = 1)

In [13]:
WO['Web Width 2'] = WO['Paper Code 2'].apply(get_web_width2)

In [14]:
WO['Construction'] = WO['Construction'].apply(cleanConstruction)
WO['Paper Code 2'] = WO.apply(lambda x: str(x['Paper Code 2'])+' SUB*' if (len(x['Construction'].split('/'))<3) and (x['Paper Code 2']!=np.nan) else x['Paper Code 2'],axis = 1)

In [15]:
def get_paper_type(cons):
    paper_type = cons.split('/')[0]
    return paper_type

In [16]:
WO['Paper Type'] = WO['Construction'].apply(get_paper_type)

In [17]:
def get_paper_basis(ptype):
    basis = re.findall(r'(\d+)',ptype)
    if len(basis)==1:
        return float(basis[0])
    else:
        l = []
        for i in range(len(basis)):
            l.append(float(basis[i]))
        return l

In [18]:
WO['Paper Basis'] = WO['Paper Type'].apply(get_paper_basis)

In [19]:
WO['Poly Basis'] = WO['Construction'].str.extract('/PE(.*)')

In [20]:
def get_poly_basis(pb):
    try:
        return float(pb)
    except:
        if ('/' in list(pb))==True:
            return float(str(pb).split('/')[0])
        else:
            return float(str(pb).split(',')[0])

In [21]:
WO['Poly Basis']=WO['Poly Basis'].apply(get_poly_basis)

In [22]:
def get_total_basis(df):
    basis = 0
    if type(df['Paper Basis'])==float:
        basis +=df['Paper Basis']
    else:
        for i in df['Paper Basis']:
            basis+=i
    basis+=df['Poly Basis']
    return basis

In [23]:
WO['Total Basis'] = WO.apply(get_total_basis,axis=1)

In [24]:
def estimated_line_speed(df):
    try:
        width = max(df['Web Width 1'],df['Web Width 2'])
        return df['Kg/Hr']/df['Total Basis']*1000/width/60
    except:
        return 0

In [25]:
WO['Estimated Line Speed'] = WO.apply(estimated_line_speed, axis = 1)

In [26]:
def resin_waste(df):
    r_ratio = df['Poly Basis']/df['Total Basis']
    return round(df['Resins Usage']-r_ratio*df['Total Production'],2)

In [27]:
def paper_waste(df):
    p = 0
    for i in df['Paper Basis']:
        p+=i
    p_ratio = p/df['Total Basis']
    return round(df['Paper Usage']-p_ratio*df['Total Production'],2)

In [28]:
mean_values = pd.DataFrame(WO.groupby(['Order','Product Code'], sort = False)['Kg/Hr','Run Time'].mean()).reset_index()
max_dict = dict(mean_values.groupby('Product Code',sort = False)['Kg/Hr'].max())

In [29]:
min_dict = dict(mean_values.groupby('Product Code',sort = False)['Kg/Hr'].min())

In [30]:
product_time = dict(mean_values.groupby('Product Code')['Run Time'].sum())
def AKG(df):
    return df['Kg/Hr']*df['Run Time']/product_time[df['Product Code']]

In [31]:
mean_values['Average Kg/Hr'] = mean_values.apply(AKG, axis = 1)
average_kg_dict = dict(mean_values.groupby('Product Code')['Average Kg/Hr'].sum())

In [32]:
def get_max(df):
    return max_dict[df['Product Code']]
def get_min(df):
    return min_dict[df['Product Code']]
def get_averageKgs(df):
    return average_kg_dict[df['Product Code']]

In [33]:
WO['Average Kg/Hr'] = WO.apply(get_averageKgs, axis =1)

In [34]:
WO['Max Kg/Hr'] = WO.apply(get_max, axis = 1)
WO['Min Kg/Hr'] = WO.apply(get_min, axis = 1)

In [35]:
WO['OP Run Time'] = WO['OP Run Time'].apply(lambda x: x if x>0 else 0)

In [36]:
WO['Setup Time'] = WO['Setup Time'].apply(lambda x: x if x>0 else -x)

In [37]:
WO['Maintanence Down Time'] = WO['Maintenance Down Time'].apply(lambda x: x if x>0 else -x)

In [38]:
data155 = pd.read_csv(r'C:\Users\bzhang\Desktop\Projects\CoatingDataManager\155 Blender Data1.csv')
data154 = pd.read_csv(r'C:\Users\bzhang\Desktop\Projects\CoatingDataManager\154 Blender Data1.csv')
data151 = pd.read_csv(r'C:\Users\bzhang\Desktop\Projects\CoatingDataManager\151 Blender Data1.csv')

In [39]:
data155= data155.dropna(axis = 0)
data154= data154.dropna(axis = 0)
data151= data151.dropna(axis = 0)

In [40]:
NWO = WO
def mean(series):
    return series.mean()

NWO['Setup Time'].fillna(0, inplace = True)
NWO['OP Run Time'].fillna(method = 'bfill', inplace =True)
NWO['Maintenance Down Time'].fillna(0, inplace = True)
NWO['Run Time'].fillna(method = 'ffill', inplace = True)
NWO['Kg/Hr'].fillna(method = 'ffill', inplace = True)
NWO['Efficiency'].fillna(method = 'ffill', inplace = True)
WO_155 =NWO[NWO['Machine']=='155'].sort_values('Production Date')
WO_154 =NWO[NWO['Machine']=='154'].sort_values('Production Date')
WO_151 =NWO[NWO['Machine']=='151'].sort_values('Production Date')
WO_155['Expected Kg/Hr'] = NWO['Kg/Hr']/NWO['Efficiency']*100
WO_154['Expected Kg/Hr'] = NWO['Kg/Hr']/NWO['Efficiency']*100
WO_151['Expected Kg/Hr'] = NWO['Kg/Hr']/NWO['Efficiency']*100
def to_hours(time):
    return round(time/pd.Timedelta(hours=1),2)
WO_155['Time Per Roll'] = WO_155['Production Date'].diff().apply(to_hours)
WO_154['Time Per Roll'] = WO_154['Production Date'].diff().apply(to_hours)
WO_151['Time Per Roll'] = WO_151['Production Date'].diff().apply(to_hours)
def setup_time(df):
    setup1 = pd.DataFrame(df.groupby(['Order','Operator'], sort = False)['Setup Time'].sum())
    setup2 = pd.DataFrame(df.groupby(['Order','Operator'], sort = False)['Setup Time'].size())
    setup2 = setup2.rename(columns = {'Setup Time':'Setup Size'})
    setupfinal = pd.DataFrame(setup1['Setup Time']/setup2['Setup Size'], columns = ['Setup Time']).reset_index()
    return setupfinal

def monthly(time):
    return str(time)[:7]
def just_date(time):
    return str(time)[:10]

setup155 = setup_time(WO_155)
setup154 = setup_time(WO_154)
setup151 = setup_time(WO_151)

setup_dict155 =  dict(zip(zip(setup155['Order'],setup155['Operator']),setup155['Setup Time']))
setup_dict154 =  dict(zip(zip(setup154['Order'],setup154['Operator']),setup154['Setup Time']))
setup_dict151 =  dict(zip(zip(setup151['Order'],setup151['Operator']),setup151['Setup Time']))

def get_setup_time155(df):
    try:
        return setup_dict155[(df['Order'],df['Operator'])]
    except:
        return 0
def get_setup_time154(df):
    try:
        return setup_dict154[(df['Order'],df['Operator'])]
    except:
        return 0
def get_setup_time151(df):
    try:
        return setup_dict151[(df['Order'],df['Operator'])]
    except:
        return 0

WO_155['Set Up Time'] = WO_155.apply(get_setup_time155, axis = 1)
WO_154['Set Up Time'] = WO_154.apply(get_setup_time154, axis = 1)
WO_151['Set Up Time'] = WO_151.apply(get_setup_time151, axis = 1)

def meantime(df):
    s1 = pd.Series(df.groupby(['Order'],sort = False)['Run Time'].apply(mean))
    s2 = pd.Series(df.groupby(['Order'],sort = False)['Qty'].size())
    s3 = pd.Series(s1/s2)
    return dict(s3)
mean155 = meantime(WO_155)
mean154 = meantime(WO_154)
mean151 = meantime(WO_151)
def get_mean155(order):
    return mean155[order]
def get_mean154(order):
    return mean154[order]
def get_mean151(order):
    return mean151[order]
WO_155['Mean Time'] = WO_155['Order'].apply(get_mean155)
WO_154['Mean Time'] = WO_154['Order'].apply(get_mean154)
WO_151['Mean Time'] = WO_151['Order'].apply(get_mean151)
WO_155['Time Per Roll'] = WO_155.apply(lambda x: x['Mean Time'] if x['Time Per Roll']>=(x['Mean Time']+12) else x['Time Per Roll'], axis = 1)
WO_154['Time Per Roll'] = WO_154.apply(lambda x: x['Mean Time'] if x['Time Per Roll']>=(x['Mean Time']+12) else x['Time Per Roll'], axis = 1)
WO_151['Time Per Roll'] = WO_151.apply(lambda x: x['Mean Time'] if x['Time Per Roll']>=(x['Mean Time']+12) else x['Time Per Roll'], axis = 1)

In [41]:
WO_155.head()

,Machine,Order,Product Code,Production Date,Qty,Run Time,OP Run Time,Setup Time,Maintenance Down Time,Operator,...,Total Basis,Estimated Line Speed,Average Kg/Hr,Max Kg/Hr,Min Kg/Hr,Maintanence Down Time,Expected Kg/Hr,Time Per Roll,Set Up Time,Mean Time
0,155,27344129,18FG-REY-07602,2018-01-02 00:06:00,340,47.5,24.0,0.0,0.0,09-0018,...,67.0,170.441593,1398.19875,2064.612903,1047.0,NaN,1412.222222,NaN,0.0,0.349265
1,155,27344129,18FG-REY-07602,2018-01-02 00:06:00,340,47.5,24.0,0.0,0.0,09-0018,...,67.0,170.441593,1398.19875,2064.612903,1047.0,NaN,1412.222222,0.00,0.0,0.349265
2,155,27344129,18FG-REY-07602,2018-01-02 00:06:00,340,47.5,24.0,0.0,0.0,09-0018,...,67.0,170.441593,1398.19875,2064.612903,1047.0,NaN,1412.222222,0.00,0.0,0.349265
3,155,27344129,18FG-REY-07602,2018-01-02 00:06:00,340,47.5,24.0,0.0,0.0,09-0018,...,67.0,170.441593,1398.19875,2064.612903,1047.0,NaN,1412.222222,0.00,0.0,0.349265
5,155,27344129,18FG-REY-07602,2018-01-02 01:02:00,323,47.5,24.0,0.0,0.0,09-0018,...,67.0,170.441593,1398.19875,2064.612903,1047.0,NaN,1412.222222,0.93,0.0,0.349265


In [42]:
def convert(df):
    try:
        return float(df['Web Width 1'])
    except:
        return float(re.findall(r"-(\d+)",df['Product Code'])[0][3::])/1000

In [43]:
machinedata=[WO_151,WO_154,WO_155]
for d in machinedata:
    d['Web Width 1']=d.apply(convert,axis =1)

In [44]:
data155.Time =  pd.to_datetime(data155.Time)
OrderStart155 = pd.DataFrame(WO_155.groupby(['Order'], sort = False)['Production Date'].min())
OrderStart155 = OrderStart155.rename(columns = {'Production Date':'Order Start'})
OrderEnd155 = pd.DataFrame(WO_155.groupby(['Order'], sort = False)['Production Date'].max())
OrderEnd155 = OrderEnd155.rename(columns = {'Production Date':'Order End'})
WO_155['Web Width 1']=WO_155['Web Width 1'].astype(float)
width155 = pd.DataFrame(WO_155.groupby(['Order'],sort = False)['Web Width 1'].mean())
OrderTime155 = pd.concat([OrderStart155,OrderEnd155,width155],sort = False,axis =1).reset_index()
OrderStart155 = dict(zip(OrderTime155['Order'],OrderTime155['Order Start']))
OrderEnd155 = dict(zip(OrderTime155['Order'],OrderTime155['Order End']))

data154.Time =  pd.to_datetime(data154.Time)
OrderStart154 = pd.DataFrame(WO_154.groupby(['Order'], sort = False)['Production Date'].min())
OrderStart154 = OrderStart154.rename(columns = {'Production Date':'Order Start'})
OrderEnd154 = pd.DataFrame(WO_154.groupby(['Order'], sort = False)['Production Date'].max())
OrderEnd154 = OrderEnd154.rename(columns = {'Production Date':'Order End'})
WO_154['Web Width 1']=WO_154['Web Width 1'].astype(float)
width154 = pd.DataFrame(WO_154.groupby(['Order'],sort = False)['Web Width 1'].mean())
OrderTime154 = pd.concat([OrderStart154,OrderEnd154,width154],sort = False,axis =1).reset_index()
OrderStart154 = dict(zip(OrderTime154['Order'],OrderTime154['Order Start']))
OrderEnd154 = dict(zip(OrderTime154['Order'],OrderTime154['Order End']))

data151.Time =  pd.to_datetime(data151.Time)
OrderStart151 = pd.DataFrame(WO_151.groupby(['Order'], sort = False)['Production Date'].min())
OrderStart151 = OrderStart151.rename(columns = {'Production Date':'Order Start'})
OrderEnd151 = pd.DataFrame(WO_151.groupby(['Order'], sort = False)['Production Date'].max())
OrderEnd151 = OrderEnd151.rename(columns = {'Production Date':'Order End'})
WO_151['Web Width 1']=WO_151['Web Width 1'].astype(float)
width151 = pd.DataFrame(WO_151.groupby(['Order'],sort = False)['Web Width 1'].mean())
OrderTime151 = pd.concat([OrderStart151,OrderEnd151,width151],sort = False,axis =1).reset_index()
OrderStart151 = dict(zip(OrderTime151['Order'],OrderTime151['Order Start']))
OrderEnd151 = dict(zip(OrderTime151['Order'],OrderTime151['Order End']))

In [45]:
data155=data155.drop(data155.loc[data155['LineSpeed']<=25].index)
data154=data154.drop(data154.loc[data154['LineSpeed']<=25].index)
data151=data151.drop(data151.loc[data151['LineSpeed']<=25].index)

In [46]:
def get_actual_resinGSM155(df):
    return data155.loc[(data155.Time >= df['Order Start']) & (data155.Time<= df['Order End'])]['LineSpeed'].mode()
def get_actual_resinGSM154(df):
    return data154.loc[(data154.Time >= df['Order Start']) & (data154.Time<= df['Order End'])]['LineSpeed'].mode()
def get_actual_resinGSM151(df):
    return data151.loc[(data151.Time >= df['Order Start']) & (data151.Time<= df['Order End'])]['LineSpeed'].mode()

In [47]:
LineSpeed155 = OrderTime155.apply(get_actual_resinGSM155,axis =1)
LineSpeed154 = OrderTime154.apply(get_actual_resinGSM154,axis =1)
LineSpeed151 = OrderTime151.apply(get_actual_resinGSM151,axis =1)

In [48]:
OrderTime155['Line Speed'] = LineSpeed155.iloc[:,0]
OrderTime154['Line Speed'] = LineSpeed154.iloc[:,0]
OrderTime151['Line Speed'] = LineSpeed151.iloc[:,0]

In [49]:
def gsm_calc(data):
    width = 1.397
    gsm = data.iloc[:,1:7].diff(axis = 0)/5/width*1000
    gsm['LineSpeed']=data['LineSpeed'].replace(0,5)
    gsm.iloc[:,:6] = gsm.iloc[:,:6].div(gsm.LineSpeed, axis = 0)
    gsm = gsm.dropna(axis = 0)
    col_names = gsm.columns.values.tolist()
    for i in range(0,6):
        col_names[i]+= ' gsm'
    col_dict = dict(zip(gsm.columns.values.tolist(),col_names))
    gsm = gsm.rename(columns = col_dict)
    gsm['4.5 Total gsm'] = gsm.iloc[:,0:3].sum(axis = 1)
    gsm['2.5 Total gsm'] = gsm.iloc[:,3:6].sum(axis = 1)
    gsm['Total gsm'] = gsm['4.5 Total gsm']+gsm['2.5 Total gsm']
    gsm['Time'] = data.Time
    return gsm

def gsm_calc_151(data):
    width = 1.397
    gsm = data.iloc[:,1:4].diff(axis = 0)/5/width*1000
    gsm['LineSpeed']=data['LineSpeed'].replace(0,5)
    gsm.iloc[:,:3] = gsm.iloc[:,:3].div(gsm.LineSpeed, axis = 0)
    gsm = gsm.dropna(axis = 0)
    col_names = gsm.columns.values.tolist()
    for i in range(0,3):
        col_names[i]+= ' gsm'
    col_dict = dict(zip(gsm.columns.values.tolist(),col_names))
    gsm = gsm.rename(columns = col_dict)
    gsm['Total gsm'] = gsm.iloc[:,0:3].sum(axis = 1)
    gsm['Time'] = data.Time
    return gsm

In [50]:
gsm155 = gsm_calc(data155)
gsm155 = gsm155.drop(gsm155.loc[gsm155['LineSpeed']<=20].index)
gsm155 = gsm155.drop(gsm155.loc[gsm155['Total gsm']<0].index)
gsm154 = gsm_calc(data154)
gsm154 = gsm154.drop(gsm154.loc[gsm154['LineSpeed']<=20].index)
gsm154 = gsm154.drop(gsm154.loc[gsm154['Total gsm']<0].index)
gsm151 = gsm_calc_151(data151)
gsm151 = gsm151.drop(gsm151.loc[gsm151['LineSpeed']<=20].index)
gsm151 = gsm151.drop(gsm151.loc[gsm151['Total gsm']<0].index)

In [51]:
def get_actual_gsm155(df):
    return gsm155.loc[(gsm155.Time >= df['Order Start']) & (gsm155.Time<= df['Order End'])]['Total gsm'].mean()/df['Web Width 1']*1.397
def get_actual_gsm154(df):
    return gsm154.loc[(gsm154.Time >= df['Order Start']) & (gsm154.Time<= df['Order End'])]['Total gsm'].mean()/df['Web Width 1']*1.397
def get_actual_gsm151(df):
    return gsm151.loc[(gsm151.Time >= df['Order Start']) & (gsm151.Time<= df['Order End'])]['Total gsm'].mean()/df['Web Width 1']*1.397

In [52]:
OrderTime155['Actual GSM'] = OrderTime155.apply(get_actual_gsm155,axis = 1)
OrderTime154['Actual GSM'] = OrderTime154.apply(get_actual_gsm154,axis = 1)
OrderTime151['Actual GSM'] = OrderTime151.apply(get_actual_gsm151,axis = 1)

In [53]:
LineSpeedDict155 = dict(zip(OrderTime155['Order'],OrderTime155['Line Speed']))
LineSpeedDict154 = dict(zip(OrderTime154['Order'],OrderTime154['Line Speed']))
LineSpeedDict151 = dict(zip(OrderTime151['Order'],OrderTime151['Line Speed']))

In [54]:
def get_real_linespeed155(order):
    return LineSpeedDict155[order]
def get_real_linespeed154(order):
    return LineSpeedDict154[order]
def get_real_linespeed151(order):
    return LineSpeedDict151[order]

In [55]:
WO_155['Line Speed'] = WO_155['Order'].apply(get_real_linespeed155)
WO_154['Line Speed'] = WO_154['Order'].apply(get_real_linespeed154)
WO_151['Line Speed'] = WO_151['Order'].apply(get_real_linespeed151)

In [56]:
gsmDict155 = dict(zip(OrderTime155['Order'],OrderTime155['Actual GSM']))
gsmDict154 = dict(zip(OrderTime154['Order'],OrderTime154['Actual GSM']))
gsmDict151 = dict(zip(OrderTime151['Order'],OrderTime151['Actual GSM']))

In [57]:
def get_real_gsm155(order):
    return gsmDict155[order]
def get_real_gsm154(order):
    return gsmDict154[order]
def get_real_gsm151(order):
    return gsmDict151[order]

In [58]:
WO_155['Coating GSM'] = WO_155['Order'].apply(get_real_gsm155)
WO_154['Coating GSM'] = WO_154['Order'].apply(get_real_gsm154)
WO_151['Coating GSM'] = WO_151['Order'].apply(get_real_gsm151)

In [59]:
WO_155['Total Down Time'] =(1-WO_155['Estimated Line Speed']/WO_155['Line Speed'])*WO_155['Run Time']
WO_154['Total Down Time'] =(1-WO_154['Estimated Line Speed']/WO_154['Line Speed'])*WO_154['Run Time']
WO_151['Total Down Time'] =(1-WO_151['Estimated Line Speed']/WO_151['Line Speed'])*WO_151['Run Time']

In [60]:
WO_155['Date'] = WO_155['Production Date'].apply(just_date)
WO_154['Date'] = WO_154['Production Date'].apply(just_date)
WO_151['Date'] = WO_151['Production Date'].apply(just_date)

In [61]:
WO_155['Over Coating'] = (WO_155['Coating GSM'] - WO_155['Poly Basis'])/WO_155['Total Basis']*WO_155['Total Production']
WO_154['Over Coating'] = (WO_154['Coating GSM'] - WO_154['Poly Basis'])/WO_154['Total Basis']*WO_154['Total Production']
WO_151['Over Coating'] = (WO_151['Coating GSM'] - WO_151['Poly Basis'])/WO_151['Total Basis']*WO_151['Total Production']

In [62]:
def get_new_df(df):
    df['Paper Type'] = df['Paper Type'].astype(str)
    df['Resins'] = df['Resins'].astype(str)
    slitter_op = ['09-0089','09-0120','09-0084','09-0213','09-0622','09-0561']
    setup1 = pd.DataFrame(df.groupby(['Order','Operator','Product Code','Date'], sort = False)['Setup Time'].sum())
    setup2 = pd.DataFrame(df.groupby(['Order','Operator','Product Code','Date'], sort = False)['Setup Time'].size())
    setup2 = setup2.rename(columns = {'Setup Time':'Setup Size'})
    setupfinal = pd.DataFrame(setup1['Setup Time']/setup2['Setup Size'], columns = ['Setup Time'])
    setuptime=[]
    for Order,new_df in setupfinal.groupby(level = 0,sort = False):
        dup=new_df['Setup Time'].duplicated()
        new_df['New Setup Time']=new_df['Setup Time'].where(~dup,0)
        for i in new_df['New Setup Time'].tolist():
            setuptime.append(i)
    setuptime = pd.Series(setuptime)
    d1 = pd.DataFrame(df.groupby(['Machine','Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'], sort = False)['Time Per Roll','Qty'].sum())
    d2 = pd.DataFrame(df.groupby(['Machine','Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'], sort = False)['Time Per Roll','Qty'].size())
    ndf = pd.concat([d1,d2], axis = 1).reset_index()
    ndf['Setup Time'] = setuptime
    ndf['Setup Time'].fillna(0, inplace = True)
    ndf = ndf.rename(columns ={'Time Per Roll':'Run Time',0:'Order Size'})
    d3 = pd.DataFrame(df.loc[df['Time Per Roll']==0].groupby(['Order','Operator','Date'], sort = False)['Time Per Roll'].size()).reset_index()
    d3 =d3.rename(columns = {'Time Per Roll':'Excess Size'})
    keys = zip(d3['Order'],d3['Operator'],d3['Date'])
    ex_size = dict(zip(keys,d3['Excess Size']))
    def excsize(df):
        if ((str(df['Order']),str(df['Operator']),str(df['Date'])) in list(ex_size.keys()))==True:
            return ex_size[(str(df['Order']),str(df['Operator']),str(df['Date']))]
        else:
            return 0
    ndf['Excess Size'] = ndf.apply(excsize,axis = 1)
    ndf['Order Size'] = ndf['Order Size']- ndf['Excess Size']
    ndf['Mean Time'] = ndf['Run Time']/ndf['Order Size']
    ndf['Run Time']= ndf.apply(lambda x: x['Mean Time']*(x['Excess Size']+1) if x['Run Time']<0.5 else x['Run Time'], axis =1)
    ndf = ndf.drop(ndf.loc[ndf['Run Time']<0.2].index)
    ndf['Run Time']= ndf.apply(lambda x: 1.2 if x['Run Time']<0.5 else x['Run Time'], axis = 1)
    Exp = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'], sort = False)['Expected Kg/Hr'].apply(mean)).reset_index()
    ndf['Expected Kg/Hr'] = round(Exp['Expected Kg/Hr'],2)
    maintain = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'], sort = False)['Maintenance Down Time'].apply(mean)).reset_index()
    ndf['Kg/Hr'] = round(ndf['Qty']/ndf['Run Time'],2)
    ndf['Efficiency'] = round(ndf['Kg/Hr']/ndf['Expected Kg/Hr']*100,2)
    ndf['Maintenance Down Time'] = maintain['Maintenance Down Time']
    ndf['Poly Basis'] = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'],sort = False)['Poly Basis'].mean()).reset_index()['Poly Basis']
    df['Paper Basis'] = df['Paper Basis'].apply(lambda x: float(x) if type(x)==float else float(max(x)))
    ndf['Paper Basis'] = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'],sort = False)['Paper Basis'].mean()).reset_index()['Paper Basis']
    ndf['Total Basis'] = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'],sort = False)['Total Basis'].mean()).reset_index()['Total Basis']
    ndf['Over Coating'] = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'],sort = False)['Over Coating'].mean()).reset_index()['Over Coating']
    ndf['Line Speed'] = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'],sort = False)['Line Speed'].mean()).reset_index()['Line Speed']
    ndf['Web Width'] = pd.DataFrame(df.groupby(['Order','Operator','Date','Product Code','Resins','Paper Code 1','Paper Type','Construction'],sort = False)['Web Width 1'].mean()).reset_index()['Web Width 1']
    
    ndf['Total Down Time'] = (1-ndf['Kg/Hr']/ndf['Total Basis']/ndf['Web Width']/60*1000/ndf['Line Speed'])*ndf['Run Time']
    ndf['Total Down Time'] = ndf['Total Down Time'].apply(lambda x: 0 if x <0 else x)
    def rewind(Operator):
        if (str(Operator) in slitter_op) == True:
            return 1
        else:
            return 0
    ndf = ndf.rename(columns = {'Date':'Production Date'})
    ndf['Rewinds'] = ndf['Operator'].apply(rewind)
    ndf = ndf.drop(ndf[ndf['Rewinds']==1].index)
    ndf = ndf.drop(columns = ['Rewinds'])
    return ndf

In [63]:
df155 = get_new_df(WO_155)
df154 = get_new_df(WO_154)
df151 = get_new_df(WO_151)
DF_corrected = pd.concat([df155,df154,df151], axis = 0)
DF_corrected = DF_corrected.drop(columns = ['Expected Kg/Hr','Excess Size','Order Size','Mean Time'])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [64]:
finalDataFrame = pd.concat([WO_155,WO_154,WO_151], axis = 0)

In [65]:
finalDataFrame['Total Down Time'] = finalDataFrame['Total Down Time'].apply(lambda x: 0 if x<0 else x)

In [66]:
finalDataFrame = finalDataFrame.drop(columns = ['Total Basis', 'Estimated Line Speed', 'Expected Kg/Hr', 'Mean Time','Date'])

In [67]:
finalDataFrame.to_csv(r'C:\Users\bzhang\Desktop\Projects\CoatingDataManager\DataFrameMain1.csv')

In [68]:
StandardGroups = pd.read_csv(r'C:\Users\bzhang\Desktop\Projects\CoatingDataManager\CtgCleanUp.csv')

In [69]:
StandardGroups['pt_group']=StandardGroups['pt_group'].apply(lambda x: 'OTHER' if (x in ['Delete','DELETE',np.nan,'oTHER'])==True else x)

In [70]:
GroupDict = dict(zip(StandardGroups['pt_part'],StandardGroups['pt_group']))

In [71]:
def get_group(code):
    try:
        return GroupDict[code]
    except:
        return 'OTHER'

In [72]:
finalDataFrame['Product Group'] = finalDataFrame['Product Code'].apply(get_group)

In [73]:
DF_corrected['Product Group']=DF_corrected['Product Code'].apply(get_group)

In [74]:
finalDataFrame.to_csv('DataFrameMain.csv')
DF_corrected.to_csv('DataFrameMod.csv')

In [75]:
DF_corrected[(DF_corrected['Machine']=='151')&(DF_corrected['Production Date']>='2019-08-01')]

,Machine,Order,Operator,Production Date,Product Code,Resins,Paper Code 1,Paper Type,Construction,Run Time,...,Efficiency,Maintenance Down Time,Poly Basis,Paper Basis,Total Basis,Over Coating,Line Speed,Web Width,Total Down Time,Product Group
1016,151,31069776,09-0494,2019-08-01,18FG-BOC-08599,"['LDPE', 'EXXONECD703', 'DOW-662']",18NK00-0400940,40NK,40NK/PE8,6.650000,...,59.27,0.0,8.0,40.0,48.0,NaN,NaN,0.940,NaN,OTHER
1017,151,31069776,09-0485,2019-08-01,18FG-BOC-08599,"['LDPE', 'EXXONECD703', 'DOW-662']",18NK00-0400940,40NK,40NK/PE8,3.480000,...,55.41,0.0,8.0,40.0,48.0,NaN,NaN,0.940,NaN,OTHER
1018,151,31223170,09-0494,2019-08-07,18FG-BUL-08922,['LDPE'],09NK00-0800880,80NK,80NK/PE16.5,1.150000,...,32.62,0.0,16.5,80.0,96.5,NaN,NaN,0.880,NaN,OTHER
1019,151,31195857,09-0494,2019-08-07,18FG-BUL-08918,['LDPE'],09NK00-0800680,80NK,80NK/PE16.5,8.900000,...,26.77,0.0,16.5,80.0,96.5,NaN,NaN,0.680,NaN,OTHER
1020,151,31120121,09-0368,2019-08-07,18FG-USG-04056,"['LDPE', 'CHILLRELEASE']",18NK00-0571335-45,57NK,57NK/PE8,4.020000,...,17.55,0.0,8.0,57.0,65.0,NaN,NaN,1.335,NaN,RELPAP
1021,151,31120121,09-0368,2019-08-08,18FG-USG-04056,"['LDPE', 'CHILLRELEASE']",18NK00-0571335-45,57NK,57NK/PE8,4.750000,...,111.68,0.0,8.0,57.0,65.0,NaN,NaN,1.335,NaN,RELPAP
1022,151,31120121,09-0494,2019-08-08,18FG-USG-04056,"['LDPE', 'CHILLRELEASE']",18NK00-0571335-45,57NK,57NK/PE8,13.070000,...,81.16,0.0,8.0,57.0,65.0,NaN,NaN,1.335,NaN,RELPAP
1023,151,31120121,09-0368,2019-08-12,18FG-USG-04056,"['LDPE', 'CHILLRELEASE']",18NK00-0571335-45,57NK,57NK/PE8,3.592105,...,77.47,0.0,8.0,57.0,65.0,NaN,NaN,1.335,NaN,RELPAP
1024,151,31254069,09-0494,2019-08-12,18FG-USC-06725,"['LDPE', 'BLUE-DIL', 'CHILLRELEASE']",18NPR00-0451345,45NPR,45NPR/PE10,9.370000,...,44.10,0.0,10.0,45.0,55.0,NaN,NaN,1.345,NaN,MASKNEWS
1025,151,31254069,09-0494,2019-08-13,18FG-USC-06725,"['LDPE', 'BLUE-DIL', 'CHILLRELEASE']",18NPR00-0451345,45NPR,45NPR/PE10,8.590000,...,91.78,0.0,10.0,45.0,55.0,NaN,NaN,1.345,NaN,MASKNEWS


In [76]:
WO_151

,Machine,Order,Product Code,Production Date,Qty,Run Time,OP Run Time,Setup Time,Maintenance Down Time,Operator,...,Maintanence Down Time,Expected Kg/Hr,Time Per Roll,Set Up Time,Mean Time,Line Speed,Coating GSM,Total Down Time,Date,Over Coating
156,151,27234277,18FG-NOV-08595,2018-01-03 09:04:00,609,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,NaN,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
157,151,27234277,18FG-NOV-08595,2018-01-03 10:11:00,641,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,1.120000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
158,151,27234277,18FG-NOV-08595,2018-01-03 10:43:00,550,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.530000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
159,151,27234277,18FG-NOV-08595,2018-01-03 11:09:00,760,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.430000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
160,151,27234277,18FG-NOV-08595,2018-01-03 11:50:00,637,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.680000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
161,151,27234277,18FG-NOV-08595,2018-01-03 12:11:00,794,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.350000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
162,151,27234277,18FG-NOV-08595,2018-01-03 12:45:00,566,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.570000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
163,151,27234277,18FG-NOV-08595,2018-01-03 13:15:00,707,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.500000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
164,151,27234277,18FG-NOV-08595,2018-01-03 13:43:00,680,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.470000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN
165,151,27234277,18FG-NOV-08595,2018-01-03 14:13:00,689,11.75,7.50,0.00,0.0,09-0485,...,NaN,1150.632911,0.500000,0.00,1.068182,NaN,NaN,NaN,2018-01-03,NaN


In [77]:
WO_151[(WO_151['Machine']=='151')&(WO_151['Production Date']>='2019-08-01')]

,Machine,Order,Product Code,Production Date,Qty,Run Time,OP Run Time,Setup Time,Maintenance Down Time,Operator,...,Maintanence Down Time,Expected Kg/Hr,Time Per Roll,Set Up Time,Mean Time,Line Speed,Coating GSM,Total Down Time,Date,Over Coating
31103,151,31069776,18FG-BOC-08599,2019-08-01 00:22:00,997,12.00,10.00,0.75,0.0,09-0494,...,NaN,1012.280702,1.72,0.75,1.500000,NaN,NaN,NaN,2019-08-01,NaN
31104,151,31069776,18FG-BOC-08599,2019-08-01 01:58:00,996,12.00,10.00,0.75,0.0,09-0494,...,NaN,1012.280702,1.60,0.75,1.500000,NaN,NaN,NaN,2019-08-01,NaN
31105,151,31069776,18FG-BOC-08599,2019-08-01 02:30:00,290,12.00,10.00,0.75,0.0,09-0494,...,NaN,1012.280702,0.53,0.75,1.500000,NaN,NaN,NaN,2019-08-01,NaN
31106,151,31069776,18FG-BOC-08599,2019-08-01 03:39:00,708,12.00,10.00,0.75,0.0,09-0494,...,NaN,1012.280702,1.15,0.75,1.500000,NaN,NaN,NaN,2019-08-01,NaN
31107,151,31069776,18FG-BOC-08599,2019-08-01 05:18:00,999,12.00,10.00,0.75,0.0,09-0494,...,NaN,1012.280702,1.65,0.75,1.500000,NaN,NaN,NaN,2019-08-01,NaN
31108,151,31069776,18FG-BOC-08599,2019-08-01 06:57:00,995,12.00,2.00,0.00,0.0,09-0485,...,NaN,1012.280702,1.65,0.00,1.500000,NaN,NaN,NaN,2019-08-01,NaN
31109,151,31069776,18FG-BOC-08599,2019-08-01 08:47:00,957,12.00,2.00,0.00,0.0,09-0485,...,NaN,1012.280702,1.83,0.00,1.500000,NaN,NaN,NaN,2019-08-01,NaN
31110,151,31332688,18HPK02-0880980 D,2019-08-01 11:07:00,1044,5.50,5.50,0.50,0.0,09-0485,...,NaN,1203.125000,2.33,0.50,1.375000,NaN,NaN,NaN,2019-08-01,NaN
31111,151,31332688,18HPK02-0880980 D,2019-08-01 13:00:00,1054,5.50,5.50,0.50,0.0,09-0485,...,NaN,1203.125000,1.88,0.50,1.375000,NaN,NaN,NaN,2019-08-01,NaN
31112,151,31332688,18HPK02-0880980 D,2019-08-01 13:06:00,1074,5.50,5.50,0.50,0.0,09-0485,...,NaN,1203.125000,0.10,0.50,1.375000,NaN,NaN,NaN,2019-08-01,NaN
